# Preliminary code usage

Highly suggest to use a venv to code this project

To create your venv, install venv if necessary:

```bash
mkdir .venv 
python3 -m venv .venv
source .venv/bin/activate
```

To install the requirements:

```bash
pip install -r requirements.txt
```


Make sure to use .venv python as kernel in jupyter notebook

```bash

## Import libraries

In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pyActigraphy


### Magnitude

In [9]:
data_folder = '../raw_data/'

df = pd.read_csv(data_folder + '1_AHA_RAW.csv')

In [10]:
magnitude_D = np.sqrt(np.square(df['x_D']) + np.square(df['y_D']) + np.square(df['z_D'])) #mano dominante
magnitude_ND = np.sqrt(np.square(df['x_ND']) + np.square(df['y_ND']) + np.square(df['z_ND'])) #mano non dominante

### Operations with time-series' magnitude
- concatenation
- difference (btw domaninat and non dominant)
- asymmetrix index (ai)

In [11]:
#compute asimmetry index and difference given magnitude 
def elaborate_magnitude(operation_type, magnitude_D, magnitude_ND):

    elaborated_magnitude = []
    
    '''
    penso sia inutile usare la concatenazione come indice 

    if operation_type == 'concat':
        elaborated_magnitude = pd.concat([magnitude_D, magnitude_ND], ignore_index=True)
    '''

    if operation_type == 'difference':
        elaborated_magnitude = magnitude_D - magnitude_ND
    elif operation_type == 'ai':
        elaborated_magnitude = (((magnitude_D - magnitude_ND) / (magnitude_D + magnitude_ND)) * 100).fillna(0)
    else: 
        print('operation type non supportata.')
        exit(1)

    return elaborated_magnitude

In [12]:
#create window_size
def create_windows(folder, operation_type, WINDOW_SIZE):
    series = []
    y_AHA = []
    y_MACS =[]
    y = []
    metadata = pd.read_excel(folder + 'metadata2022_04.xlsx')

    for j in range (1, metadata.shape[0]+1):
        df = pd.read_csv(folder + 'data/' + str(j) + '_AHA_1sec.csv')

        #print('Paziente ' + str(j) + ' -> df.shape[0] = ' + str(df.shape[0]))

        # Nel caso in cui non bastasse una duplicazione dell'intera time series questa verrà scartata
        if df.shape[0]<WINDOW_SIZE:
            df_concat = pd.concat([df, df.iloc[:WINDOW_SIZE-df.shape[0]]], ignore_index = True, axis = 0)
            df = df_concat
            #print('MODIFICATO Paziente ' + str(j) + ' -> df.shape[0] = ' + str(df.shape[0]))

        scart = (df.shape[0] % WINDOW_SIZE)/2
        
        df_cut = df.iloc[math.ceil(scart):df.shape[0]-math.floor(scart)]


        # Calculating magnitude
        magnitude_D = np.sqrt(np.square(df_cut['x_D']) + np.square(df_cut['y_D']) + np.square(df_cut['z_D']))
        magnitude_ND = np.sqrt(np.square(df_cut['x_ND']) + np.square(df_cut['y_ND']) + np.square(df_cut['z_ND']))
        for i in range (0, len(magnitude_D), WINDOW_SIZE):
            chunk_D = magnitude_D.iloc[i:i + WINDOW_SIZE]
            chunk_ND = magnitude_ND.iloc[i:i + WINDOW_SIZE]
            series.append(elaborate_magnitude(operation_type, chunk_D, chunk_ND))
            y_AHA.append(metadata['AHA'].iloc[j-1])
            y_MACS.append(metadata['MACS'].iloc[j-1])
            y.append(metadata['hemi'].iloc[j-1]-1)
    
    return np.array(series), y_AHA, y_MACS, np.array(y)

In [ ]:
# Fase di chunking (OPTIONAL)
#Need to select a window size

    for j in range (0, len(magnitude_D), window_size):

        chunk_D = magnitude_D.iloc[j:j + window_size]
        chunk_ND = magnitude_ND.iloc[j:j + window_size]

        if chunk_D.size == window_size and chunk_ND.size == window_size:
            
            for es in estimators:
                es['series'].append(elaborate_magnitude(es['method'], chunk_D, chunk_ND))

            if chunk_D.agg('sum') == 0 and chunk_ND.agg('sum') == 0:
                to_discard.append(int(j/window_size))

### Preprocessing starting from raw data

In [ ]:
# Nella tesi di triennale non fanno un cazzo di pre-processing, prendono direttamente x, y, z dalle time-series



### Correlation analysis

- correlation btw WEEK sessions and Clinical sessions (1h) --> sliding

In [ ]:
#Nella tesi di triennale usano semplicemente corrcoef ma così non fai sliding

